# Classification of documents based on their sentiment

## Obtaining the movie review dataset

In [1]:
import pyprind
import pandas as pd
from tqdm import tqdm
import numpy as np
import os
import sys
import warnings

warnings.filterwarnings('ignore')

In [2]:
basepath = ".\\data\\aclImdb_v1\\aclImdb"

In [3]:
total_files = 0
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        total_files += len(os.listdir(path))

In [4]:
total_files

50000

In [5]:
labels = {
    'pos': 1,
    'neg': 0
}

total_files = 0
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        total_files += len(os.listdir(path))

df = pd.DataFrame()

with tqdm(total=total_files, desc='Processando arquivos') as pbar:
    for s in ('test', 'train'):
        for l in ('pos', 'neg'):
            path = os.path.join(basepath, s, l)
            for file in sorted(os.listdir(path)):
                with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                    txt = infile.read()
                df_temp = pd.DataFrame([[txt, labels[l]]])
                df = pd.concat(
                    [df, df_temp],
                    ignore_index=True
                )
                pbar.update(1)

Processando arquivos:   0%|          | 173/50000 [00:02<11:13, 73.93it/s]


KeyboardInterrupt: 

In [27]:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('.\\data\\movie_data.csv', index=False, encoding='utf-8')

## Loading the dataset

In [6]:
df = pd.read_csv('.\\data\\movie_data.csv', encoding='utf-8')
df = df.rename(columns={'0': 'review', '1': 'sentiment'})
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [7]:
df.shape

(50000, 2)

## Transforming words into feature vectors

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
count = CountVectorizer(ngram_range=(1,1))

docs = np.array(
    [
        "The sun is shining",
        "The weather is sweet",
        "The sun is shining, the weather is sweet, and one and one is two"
    ]
)

bag = count.fit_transform(docs)

In [10]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [11]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


## Assessing word relevancy via term frequency-inverse document frequency

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer

In [13]:
tfidf = TfidfTransformer(
    use_idf=True,
    norm='l2',
    smooth_idf=True
)

np.set_printoptions(precision=2)

In [14]:
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


## Cleaning text data

In [15]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [16]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (
        re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    )
    return text

In [17]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [18]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [19]:
df['review'] = df['review'].apply(preprocessor)

## Processing documents into tokens

In [35]:
def tokenizer(text):
    return text.split()

In [36]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [37]:
from nltk.stem.porter import PorterStemmer

In [38]:
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [39]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [40]:
# Remove Stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\francisco.procopio\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot') if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

## Training a logistic regression model for document classification

In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [48]:
tfidf = TfidfVectorizer(
    strip_accents=None,
    lowercase=False,
    preprocessor=None
)

small_param_grid = [
    {
        'vect__ngram_range': [(1,1)],
        'vect__stop_words': [None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    },
    {
        'vect__ngram_range': [(1,1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer],
        'vect__use_idf': [False],
        'vect__norm': [None],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    }
]

In [49]:
lr_tfidf = Pipeline(
    [
        ('vect', tfidf),
        ('clf', LogisticRegression(solver='liblinear'))
    ]
)

In [50]:
gridSearch_lr_tfidf = GridSearchCV(
    lr_tfidf,
    small_param_grid,
    scoring='accuracy',
    cv=5,
    verbose=2,
    n_jobs=-1
)

In [51]:
gridSearch_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


,estimator,Pipeline(step...liblinear'))])
,param_grid,"[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'], 'vect__ngram_range': [(1, ...)], 'vect__stop_words': [None], ...}, {'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'], 'vect__ngram_range': [(1, ...)], 'vect__norm': [None], ...}]"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [52]:
print(f"Best parameter set: {gridSearch_lr_tfidf.best_params_}")

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x000001C16F07F1A0>}


In [53]:
print(f'CV Accuracy: {gridSearch_lr_tfidf.best_score_:.3f}')

CV Accuracy: 0.897


In [54]:
classifier = gridSearch_lr_tfidf.best_estimator_
print(f'Test Accuracy: {classifier.score(X_test, y_test):.3f}')

Test Accuracy: 0.899


## Working with bigger data - online algorithms and out-of-core learning

In [56]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [76]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall(
        '(?::|;|=)(?:-)?(?:\)|\(|D|P)',
        text.lower()
    )

    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]

    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [62]:
next(stream_docs(path='.\\data\\movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [79]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(
    decode_error='ignore',
    n_features=2**21,
    preprocessor=None,
    tokenizer=tokenizer
)

clf = SGDClassifier(loss='log_loss', random_state=1)

doc_stream = stream_docs(path='.\\data\\movie_data.csv')

In [80]:
from tqdm import tqdm

classes = np.array([0, 1])
total = 45
with tqdm(total=total, desc='Processando arquivos') as pbar:
    for _ in range(45):
        X_train, y_train = get_minibatch(doc_stream, size=1000)
        if not X_train:
            break
        X_train = vect.transform(X_train)
        clf.partial_fit(X_train, y_train, classes=classes)
        pbar.update(1)

Processando arquivos: 100%|██████████| 45/45 [00:13<00:00,  3.31it/s]


In [81]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print(f"Accuracy: {clf.score(X_test, y_test):.3f}")

Accuracy: 0.868


## Topic modeling with Latent Dirichlet Allocation

In [3]:
import pandas as pd

df = pd.read_csv(
    '.\\data\\movie_data.csv',
    encoding='utf-8'
)

df = df.rename(
    columns={
        "0": "review",
        "1": "sentiment"
    }
)

df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(
    stop_words='english',
    max_df=.1,
    max_features=5000
)

X = count.fit_transform(
    df['review'].values
)

In [8]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(
    n_components=10,
    random_state=123,
    learning_method='batch'
)

X_topics = lda_model.fit_transform(X)

In [9]:
lda_model.components_.shape

(10, 5000)

In [13]:
n_top_words = 5
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda_model.components_):
    print(f'======= Topic {topic_idx+1} =======')
    print(' '.join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]))

======= Topic 1 =======
horror worst script effects budget
======= Topic 2 =======
dvd watched video music guy
======= Topic 3 =======
war american series history documentary
======= Topic 4 =======
game killer murder thriller crime
======= Topic 5 =======
kids comedy episode series school
======= Topic 6 =======
family woman mother beautiful feel
======= Topic 7 =======
role performance comedy john plays
======= Topic 8 =======
action horror john effects dr
======= Topic 9 =======
book version original read music
======= Topic 10 =======
action wife father police james


In [16]:
horror = X_topics[:, 4].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print(f'========= Horror movie #{iter_idx + 1} =========')
    print(df['review'][movie_idx][:300], "...\n")

========= Horror movie #1 =========
Watching this PPV, I had high expectations for it, since Smackdown is the best show in WWE, this is a very good PPV as it is the last one before Wrestlemania.<br /><br />FIRST MATCH-HARDY BOYS & CHRIS BENOIT VS. MVP & MNM W/ MELINA IN A 6 MAN TAG TEAM MATCH Good way to start the match. It started a  ...

========= Horror movie #2 =========
*SPOILERS*<br /><br />This is only the second pay-per-view I've given a perfect 10, the first being the 1991 Royal Rumble. It was full of exciting matches that weren't memorable, just disposable fun. And that's why I love it.<br /><br />The opening match between Razor and DiBiase, as well as Ludvig  ...

========= Horror movie #3 =========
I loved this show growing up and I still watch the first season DVD at age 19 today. What can I say? I grew up in a house much like the one on Full House. I had a dad, two sisters, and a dog. I guess the only difference was that I did not live with my uncle and my dad's best frie